## Part IV.1: Experiments with CNN Model Architecture

In this section, experiments and tuning exercises will be conducted using CNN foundational model architecture.  This will include the following:

1.  Experiments with different CNN model architectures (number of conv layers, etc.)
2.  Experiments with regularization methods (if necessary) including dropout layers, batch normalization, etc.
3.  Hyperparameter tuning/optimization